<a href="https://colab.research.google.com/github/ashayustech/K-Anonymity/blob/master/Copy_of_using_transformers_for_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers


     |████████████████████████████████| 501kB 2.7MB/s 
     |████████████████████████████████| 870kB 44.2MB/s 
     |████████████████████████████████| 1.0MB 47.1MB/s 
     |████████████████████████████████| 3.7MB 32.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=b84af431e637c731bd566bf99b0a5bc5d055632eb050734c936bbc164decd5ca
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset
We'll use pandas to read the dataset and load it into a dataframe.

In [6]:
ls

'file3 (1).csv'   sample_data/


In [0]:
value=1
np.random.seed(value)

In [0]:
df = pd.read_csv('file3 (1).csv')

In [0]:
df[' Data']=df[' Data'].str.lower()

In [10]:
df.head()

,Data,class
0,joint venture partner,0.0
1,owner and principal entrepreneur,0.0
2,avionics and instrumentation engineer,0.0
3,efren gonzalez,0.0
4,director of engineering/operations,0.0


In [11]:
df.isnull().sum()

 Data    2
class    3
dtype: int64

In [12]:
df.size

2560

In [0]:
i=[df['class'].isnull()]


In [14]:
len(i[0])

1280

Listing index with nan values
Dropping the values 

In [15]:
for val in range(len(i[0])):
  if (i[0][val]==True):
    print(val)

1277
1278
1279


In [0]:
df.dropna(axis=0,inplace=True)


In [17]:
df.shape

(1277, 2)

### data is now well formatted for use



In [18]:
df.isnull().sum()

 Data    0
class    0
dtype: int64

## Listing class wise values out of 402

In [60]:
df['class'].value_counts()

0.0    830
1.0    447
Name: class, dtype: int64

#### Save for later use


In [0]:
df.to_csv('use.csv')

In [0]:
file=pd.read_csv('use.csv')

### Filtered multiple occurences of file but the class is still imbalanced


1.   Can't depend on just accuracy
2.   May need more data for minority class or use sampling technique





In [63]:
file['class'].value_counts()

0.0    830
1.0    447
Name: class, dtype: int64

In [0]:
file=df

## Parameters to trust ( *Not Accuracy*)


---


*   Precision
*   Balanced Accuracy

*   Recall 
*   F1 Score






In [0]:
batch_1=file

## Loading the Pre-trained BERT model


In [21]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [22]:
file[' Data']

0                       joint venture partner
1            owner and principal entrepreneur
2       avionics and instrumentation engineer
3                              efren gonzalez
4          director of engineering/operations
                        ...                  
1272                              conferences
1273                                  others 
1274                                    other
1275                   additional information
1276                   additional information
Name:  Data, Length: 1277, dtype: object

In [0]:
tokenized = file[' Data'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))



*   adds <cls> token

*   adds <seq> token
*   adds ## for words contained in the sentence





In [24]:
tokenized

0                           [101, 4101, 6957, 4256, 102]
1                    [101, 3954, 1998, 4054, 10670, 102]
2       [101, 20704, 3258, 6558, 1998, 16015, 3992, 102]
3                   [101, 1041, 19699, 2368, 10121, 102]
4               [101, 2472, 1997, 3330, 1013, 3136, 102]
                              ...                       
1272                                    [101, 9281, 102]
1273                                    [101, 2500, 102]
1274                                    [101, 2060, 102]
1275                              [101, 3176, 2592, 102]
1276                              [101, 3176, 2592, 102]
Name:  Data, Length: 1277, dtype: object

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [26]:
np.array(padded).shape

(1277, 11)

In [0]:
view=pd.DataFrame(padded)

## Padding is done to make all inputs of same size 

In [28]:
view.tail(10)

,0,1,2,3,4,5,6,7,8,9,10
1267,101,10618,102,0,0,0,0,0,0,0,0
1268,101,6105,1998,10618,2015,102,0,0,0,0,0
1269,101,4772,102,0,0,0,0,0,0,0,0
1270,101,4772,102,0,0,0,0,0,0,0,0
1271,101,3034,102,0,0,0,0,0,0,0,0
1272,101,9281,102,0,0,0,0,0,0,0,0
1273,101,2500,102,0,0,0,0,0,0,0,0
1274,101,2060,102,0,0,0,0,0,0,0,0
1275,101,3176,2592,102,0,0,0,0,0,0,0
1276,101,3176,2592,102,0,0,0,0,0,0,0


In [29]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1277, 11)

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [0]:
features = last_hidden_states[0][:,0,:].numpy()

In [0]:
labels = file['class']

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [34]:
 parameters = {'C': np.linspace(0.0001, 100, 20)}
 grid_search = GridSearchCV(LogisticRegression(), parameters)
 grid_search.fit(train_features, train_labels)

 print('best parameters: ', grid_search.best_params_)
 print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scrores:  0.9289212478184993


In [36]:
lr_clf = LogisticRegression(C=5.263252631578947)
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=5.263252631578947, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
lr_clf.score(test_features, test_labels)

0.9

In [38]:
len(test_features)

320

In [0]:
y_pred=lr_clf.predict(test_features)

In [40]:
len(y_pred)

320

In [41]:
test_labels

81      0.0
941     1.0
1013    1.0
1274    1.0
169     0.0
       ... 
1211    1.0
1213    1.0
541     0.0
674     0.0
819     0.0
Name: class, Length: 320, dtype: float64

In [0]:
from sklearn.metrics import classification_report as clfr

In [43]:
print(clfr(test_labels,y_pred))

              precision    recall  f1-score   support

         0.0       0.92      0.92      0.92       202
         1.0       0.86      0.86      0.86       118

    accuracy                           0.90       320
   macro avg       0.89      0.89      0.89       320
weighted avg       0.90      0.90      0.90       320



# Testing

In [0]:
list=['Renisha Alexender','Objective','Experience','Pegoli Inc']


AttributeError: ignored

In [0]:
headings=pd.DataFrame(list,columns=['heading'])

In [46]:
headings

,heading
0,Renisha Alexender
1,Objective
2,Experience
3,Pegoli Inc


In [47]:
headings.columns

Index(['heading'], dtype='object')

In [0]:

tokenization = headings['heading'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [0]:
max_len = 0
for i in tokenization.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenization.values])

In [50]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(4, 7)

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [0]:
test = last_hidden_states[0][:,0,:].numpy()

In [0]:
heading_extracted=[]

In [0]:
predictions=lr_clf.predict(test)


In [58]:
predictions[0]

0.0

In [0]:
for i in range(len(predictions)):
  if int(predictions[i])==1:
    heading_extracted.append(list[i])

In [68]:
print(list)

['Renisha Alexender', 'Objective', 'Experience', 'Pegoli Inc']


In [69]:
heading_extracted

['Objective', 'Experience']

In [0]:
def heading_predict(x):
  
  heading_extracted=[]
  headings=pd.DataFrame(x,columns=['heading'])
  tokenization = headings['heading'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  max_len = 0
  for i in tokenization.values:
    if len(i) > max_len:
        max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenization.values])
  attention_mask = np.where(padded != 0, 1, 0)
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)

  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)
  test = last_hidden_states[0][:,0,:].numpy()  
  predictions=lr_clf.predict(test)
  for i in range(len(predictions)):
    if int(predictions[i])==1:
      heading_extracted.append(list[i])
  return heading_extracted  



In [65]:
heading_predict(list)

['Objective', 'Experience']

Personal Info  0 <br>
Organization   1<br>
Education      2<br>
Skills         3<br>
Others         4




# Using better classifiers

In [0]:
!pip install simpletransformers



In [0]:
from simpletransformers.classification import ClassificationModel

In [0]:
par={'num_train_epochs': 12,'best_model_dir': 'outputs/best_model'}

In [0]:
from keras.models import load_model

In [0]:
from keras.models import Sequential

In [0]:
model = ClassificationModel('bert', 'bert-base-cased', num_labels=2,use_cuda=False)
model.train_model(train_df=train,output_dir='/nn/',eval_df=test ) 
model.save_model('f.h5')


ValueError: ignored

In [0]:
cd nn

/content/nn


In [0]:
ls

In [0]:
model.args

{'adam_epsilon': 1e-08,
 'best_model_dir': 'outputs/best_model',
 'cache_dir': 'cache_dir/',
 'do_lower_case': False,
 'early_stopping_delta': 0,
 'early_stopping_metric': 'eval_loss',
 'early_stopping_metric_minimize': True,
 'early_stopping_patience': 3,
 'encoding': None,
 'eval_batch_size': 8,
 'evaluate_during_training': False,
 'evaluate_during_training_steps': 2000,
 'evaluate_during_training_verbose': False,
 'fp16': False,
 'fp16_opt_level': 'O1',
 'gradient_accumulation_steps': 1,
 'learning_rate': 4e-05,
 'logging_steps': 50,
 'manual_seed': None,
 'max_grad_norm': 1.0,
 'max_seq_length': 128,
 'model_name': 'bert-base-cased',
 'model_type': 'bert',
 'n_gpu': 1,
 'no_cache': False,
 'num_train_epochs': 1,
 'output_dir': 'outputs/',
 'overwrite_output_dir': False,
 'process_count': 38,
 'regression': False,
 'reprocess_input_data': True,
 'save_eval_checkpoints': True,
 'save_model_every_epoch': True,
 'save_steps': 2000,
 'silent': False,
 'sliding_window': False,
 'stride':

###  Training on 300 lines and to use new lines for predicting thr performance

In [0]:
# train=pd.read_csv('train.csv')
# test=pd.read_csv('test.csv')

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train,X_test=train_test_split(df,train_size=.70)

In [0]:
train=X_train
test=X_test

In [0]:
train=pd.DataFrame(train)

In [0]:
test=pd.DataFrame(test)

In [0]:
ls

'file3 (1).csv'   sample_data/


In [0]:
!mkdir new

In [0]:
model.train_model(train_df=train,output_dir='/new',eval_df=test ) 
model.save_pretrained('/save')


Converting to features started. Cache is not used.


Running loss: 0.420863
Training of bert model complete. Saved to /new.


AttributeError: ignored

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir heading_classifier

In [0]:
cd heading_classifier/

/content/drive/My Drive/heading_classifier


In [0]:
result, model_outputs, wrong_predictions = model.eval_model(test)

Converting to features started. Cache is not used.



{'mcc': 0.6867313953952584, 'tp': 103, 'tn': 203, 'fp': 41, 'fn': 13, 'eval_loss': 0.3098130907035536}


In [0]:
result

{'eval_loss': 0.3098130907035536,
 'fn': 13,
 'fp': 41,
 'mcc': 0.6867313953952584,
 'tn': 203,
 'tp': 103}

In [0]:
model_outputs

array([[ 1.95568419e+00, -2.94334531e+00],
       [-8.44031394e-01,  1.35725808e+00],
       [-8.52615833e-01,  1.06495833e+00],
       [-8.84266555e-01,  1.06324506e+00],
       [ 2.19324994e+00, -3.01013565e+00],
       [-6.80717230e-01,  3.76625359e-01],
       [-5.32210529e-01,  1.32551134e-01],
       [ 5.67181468e-01, -1.13667071e+00],
       [ 1.44636965e+00, -2.31200099e+00],
       [ 2.01950026e+00, -2.99885941e+00],
       [ 1.68271530e+00, -2.70005989e+00],
       [-5.33708096e-01, -2.39178240e-02],
       [ 2.16635418e+00, -3.08715820e+00],
       [ 1.77049017e+00, -2.80076694e+00],
       [ 2.05453753e+00, -2.99608278e+00],
       [ 1.42313099e+00, -2.38037133e+00],
       [ 1.81092358e+00, -2.72513366e+00],
       [ 1.95817935e+00, -2.68549299e+00],
       [ 1.96125162e+00, -2.94752693e+00],
       [-3.40221137e-01, -1.32514328e-01],
       [-9.40237284e-01,  1.23969567e+00],
       [-1.10582089e+00,  1.48580945e+00],
       [ 1.44792295e+00, -2.41590405e+00],
       [ 1.

There are total 25 wrong predictions

In [0]:
len(wrong_predictions)

54

In [0]:
test.shape

(360, 2)

In [0]:
from sklearn.metrics import f1_score, accuracy_score
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

In [0]:

scores2, model_outputs, wrong_predictions = model.eval_model(test, f1=f1_multiclass, acc=accuracy_score)

Converting to features started. Cache is not used.


{'mcc': 0.6867313953952584, 'tp': 103, 'tn': 203, 'fp': 41, 'fn': 13, 'eval_loss': 0.3098130907035536, 'f1': 0.85, 'acc': 0.85}


In [0]:
heading

245

In [0]:
validate=model.predict(heading)

Converting to features started. Cache is not used.


In [0]:
index=[]
for i in range(len(validate[0])):
  if validate[0][i]==0:
    index.append(i)work history


SyntaxError: ignored

In [0]:
len(index)

45

In [0]:
heading[15]

'position'

In [0]:
index

[0,
 15,
 20,
 22,
 29,
 31,
 39,
 43,
 44,
 48,
 66,
 73,
 84,
 85,
 86,
 88,
 94,
 95,
 103,
 104,
 105,
 107,
 116,
 131,
 134,
 141,
 142,
 143,
 146,
 148,
 155,
 163,
 166,
 174,
 178,
 189,
 190,
 193,
 194,
 195,
 198,
 201,
 221,
 225,
 243]

In [0]:
wrong=[]
for val in index:
  wrong.append(heading[val])

In [0]:
wrong

['Educated',
 'position',
 'personal profile',
 'Competency Matrix',
 'professional development',
 'objective',
 'academic',
 'certificates',
 'project annexure',
 'organisational experiance',
 'conference',
 'project overview',
 'achievement',
 'professional career',
 'career track',
 'contact details',
 'organisational scan',
 'educated',
 'work history',
 'Academic',
 'technical summary',
 'projects history',
 'employment history',
 'Professional Development',
 'certification',
 'credentials',
 'Technical Skill',
 'Publication',
 'employment',
 'Skill Set',
 'army courses',
 'introduction',
 'Organisational Scan',
 'competency matrix',
 'publication',
 'license and certifications',
 'professional summary',
 'Introduction',
 'profile summary',
 'Organisational Experiance',
 'professional profile',
 'Project Annexure',
 'ACADEMIC',
 'Conference',
 'CERTIFICATES']

In [0]:
predictions, raw_output  = model.predict(['skill'])

Converting to features started. Cache is not used.


In [0]:
predictions

array([1])

In [0]:
raw_output

array([[-0.7209952 ,  0.62882924]], dtype=float32)

In [0]:
pred,raw=model.predict(['Post Graduate Diploma in Management(Systems & Marketing), New Horizon College, Bangalore, KA in 2011'])

Features loaded from cache at cache_dir/cached_dev_bert_128_5_1


In [0]:
pred

array([4])

In [0]:
raw

array([[-0.9837271 , -0.22713107, -0.76111245,  0.06992629,  0.8009652 ]],
      dtype=float32)

In [0]:
pred1,raw1=model.predict(['Mainframe (Datacom, VSAM, CA-Dispatch,Control-M, Selcopy Z/os & Z/vm, PRD1, PRDT'])

Features loaded from cache at cache_dir/cached_dev_bert_128_5_1


In [0]:
pred1

array([4])

In [0]:
raw1

array([[-0.9837271 , -0.22713107, -0.76111245,  0.06992629,  0.8009652 ]],
      dtype=float32)

In [0]:
pred2,raw2=model.predict(['dobe AEM CQx5,6.0, Adobe CRX, Apache Sling, OSGi, Fatwire Content Management System, Macromedia Contribute 3, 4, and CS3, Homesite, and custom CMS and database solutions '])

Features loaded from cache at cache_dir/cached_dev_bert_128_5_1


In [0]:
pred2,raw2

(array([4]),
 array([[-0.9837271 , -0.22713107, -0.76111245,  0.06992629,  0.8009652 ]],
       dtype=float32))

In [0]:
dict= {
        "objective": [
            "objective",
			"professional objective",
            "career objective",
			"OBJECTIVE",
            "PROFESSIONAL OBJECTIVE",
            "CAREER OBJECTIVE",
			"Objective",
            "Professional Objective",
            "Career Objective"
			
        ],
        "courses": [
            "army courses",
			"ARMY COURSES",
			"Army Courses"
        ],
        "achievement": [
            "achievement",
            "honors & activities",
			"ACHIEVEMENT",
            "HONORS & ACTIVITIES",
			"Achievement",
            "Honors & Activities"
		],
        "summary": [
            "profile summary",
            "professional summary",
            "introduction",
            "summary",
			"PROFILE SUMMARY",
            "PROFESSIONAL SUMMARY",
            "INTRODUCTION",
            "SUMMARY",
			"Profile Summary",
            "Professional Summary",
            "Introduction",
            "Summary"
        ],
        "experience": [
            "work history",
			"employment",
			"employment history",
			"organisational experiance",
			"professional accomplishments",
            "experience",
            "summary of experiance",
            "experience synopsis",
            "organisational scan",
            "professional profile",
			"professional career",
            "summary",
			"career track",
            "experience",
            "experience summary",
			"industry experience",
            "work experience",
			"professional experience",
			"professional background",
            "current experience",
            "current organizational experience",
			"WORK HISTORY",
            "EXPERIENCE",
            "SUMMARY OF EXPERIANCE",
			"INDUSTRY EXPERIENCE",
            "EXPERIENCE SYNOPSIS",
            "ORGANISATIONAL SCAN",
            "PROFESSIONAL PROFILE",
			"PROFESSIONAL ACCOMPLISHMENTS",
			"ORGANISATIONAL EXPERIANCE",
			"EMPLOYMENT",
			"EMPLOYMENT HISTORY",
			"PROFESSIONAL CAREER",
			"PROFESSIONAL BACKGROUND",
            "SUMMARY",
            "EXPERIENCE",
            "EXPERIENCE SUMMARY",
            "WORK EXPERIENCE",
			"CAREER TRACK",
			"PROFESSIONAL EXPERIENCE",
            "CURRENT EXPERIENCE",
            "CURRENT ORGANIZATIONAL EXPERIENCE",
			"Work History",
            "Experience",
            "Summary Of Experiance",
            "Experience Synopsis",
            "Organisational Scan",
            "Professional Profile",
			"Professional Career",
			"Organisational Experiance",
			"Employment",
			"Employment History",
			"Career Track",
			"Industry Experience",
			"Professional Background",
			"Professional Accomplishments",
            "Summary",
            "Experience",
            "Experience Summary",
            "Work Experience",
            "Professional Experience",
            "Current Experience",
            "Current Organizational Experience"
        ],
        "education": [
            "education",
            "educational",
            "educational qualification",
            "academic qualification",
            "qualification",
			"EDUCATION",
			"EDUCATIONAL",
            "EDUCATIONAL QUALIFICATION",
            "ACADEMIC QUALIFICATION",
            "QUALIFICATION",
			"Education",
            "Educational Qualification",
            "Academic Qualification",
            "Qualification",
			"Qualifications",
			"qualifications",
			"QUALIFICATIONS",
			"Educated",
			"EDUCATED",
			"educated",
			"Professional Development",
			"PROFESSIONAL DEVELOPMENT",
			"professional development",
			"Credentials",
			"CREDENTIALS",
			"credentials",
			"Academic",
			"ACADEMIC",
			"academic"
        ],
        "skills": [
            "additional skills",
            "skills",
            "proficiencies and specialties",
            "skill set",
            "technical skill",
            "skills summary",
            "core competencies",
            "technical summary",
            "skills set",
            "personal skills",
            "additional it’s skills",
            "competency matrix",
            "areas of expertise",
			"ADDITIONAL SKILLS",
            "SKILLS",
            "PROFICIENCIES AND SPECIALTIES",
            "SKILL SET",
            "TECHNICAL SKILL",
            "SKILLS SUMMARY",
            "CORE COMPETENCIES",
            "TECHNICAL SUMMARY",
            "SKILLS SET",
            "PERSONAL SKILLS",
            "ADDITIONAL IT’S SKILLS",
            "COMPETENCY MATRIX",
            "AREAS OF EXPERTISE",
			"Additional Skills",
            "Skills",
            "Proficiencies And Specialties",
            "Skill Set",
            "Technical Skill",
            "Skills Summary",
            "Core Competencies",
            "Technical Summary",
            "Skills Set",
            "Personal Skills",
            "Additional It’s Skills",
            "Competency Matrix",
            "Areas Of Expertise"
        ],
        "training": [
            "trainings",
			"TRAININGS",
			"Trainings",
			"training",
			"TRAINING",
			"Training"
        ],
        "projects": [
            "professional memberships",
            "projects history",
            "project overview",
            "major projects",
            "project annexure",
			"PROFESSIONAL MEMBERSHIPS",
            "PROJECTS HISTORY",
            "PROJECT OVERVIEW",
            "MAJOR PROJECTS",
            "PROJECT ANNEXURE",
			"Professional Memberships",
            "Projects History",
            "Project Overview",
            "Major Projects",
            "Project Annexure"
        ],
        "links": [
            "links",
			"LINKS",
			"Links"
        ],
        "contacts": [
            "contacts",
			"CONTACTS",
			"Contacts"
        ],
        "positions": [
            "positions",
            "position",
			"POSITIONS",
            "POSITION",
			"Positions",
            "Position"
        ],
        "personal_details": [
            "personal details",
            "personal information",
            "personal profile",
			"PERSONAL DETAILS",
            "PERSONAL INFORMATION",
            "PERSONAL PROFILE",
			"Personal Details",
            "Personal Information",
            "Personal Profile"
        ],
        "contact_details": [
            "contact details",
			 "CONTACT DETAILS",
			 "Contact Details"
        ],
        "awards": [
            "awards",
            "awards & recognitions",
            "awards and recognition",
            "awards received",
			"AWARDS",
            "AWARDS & RECOGNITIONS",
            "AWARDS AND RECOGNITION",
            "AWARDS RECEIVED",
			"Awards",
            "Awards & Recognitions",
            "Awards And Recognition",
            "Awards Received"
        ],
        "certification": [
            "certifications",
            "certificates",
            "certifications",
            "professional certifications",
            "certification",
            "license and certifications",
			"CERTIFICATIONS",
            "CERTIFICATES",
            "CERTIFICATIONS",
            "PROFESSIONAL CERTIFICATIONS",
            "CERTIFICATION",
             "LICENSE AND CERTIFICATIONS",
			 "Certifications",
            "Certificates",
            "Certifications",
            "Professional Certifications",
            "Certification",
            "License And Certifications"
        ],
        "publication": [
            "publication",
            "publications",
			"PUBLICATION",
            "PUBLICATIONS",
			 "Publication",
            "Publications"
        ],
        "conference": [
            "conference",
            "conferences",
			"CONFERENCE",
            "CONFERENCES",
			 "Conference",
            "Conferences"
        ],
        "others": [
            "other",
            "additional information",
			"OTHER",
            "ADDITIONAL INFORMATION",
			"Other",
			"Additional Information"
        ]
    }

In [0]:
titles=[]

In [0]:
for val in dict.values():
  titles.append(val)

In [0]:
heading=[]

In [0]:
for val in titles:
  for text in val:
    heading.append(text)

In [0]:
len(heading)

254

In [0]:
heading=list(set(heading))

In [0]:
heading

['Educated',
 'links',
 'education',
 'AWARDS AND RECOGNITION',
 'Professional Memberships',
 'Career Objective',
 'Professional Background',
 'Position',
 'PROJECT ANNEXURE',
 'Summary Of Experiance',
 'Additional Information',
 'Professional Certifications',
 'Qualifications',
 'ADDITIONAL INFORMATION',
 'work experience',
 'position',
 'career objective',
 'honors & activities',
 'Summary',
 'SUMMARY',
 'personal profile',
 'PROFILE SUMMARY',
 'Competency Matrix',
 'other',
 'industry experience',
 'SKILLS SUMMARY',
 'Skills Summary',
 'CONTACT DETAILS',
 'Current Experience',
 'professional development',
 'EDUCATION',
 'objective',
 'PUBLICATIONS',
 'PROFESSIONAL CERTIFICATIONS',
 'Professional Summary',
 'PROFICIENCIES AND SPECIALTIES',
 'trainings',
 'Profile Summary',
 'SKILL SET',
 'academic',
 'POSITIONS',
 'Honors & Activities',
 'TECHNICAL SKILL',
 'certificates',
 'project annexure',
 'Industry Experience',
 'SKILLS SET',
 'LINKS',
 'organisational experiance',
 'MAJOR PROJ